In [1]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
proj_dir = os.path.abspath('../..')

if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))

import numpy as np
import scipy.stats as stats
import pandas as pd
from scipy.spatial import distance
from random import random
from sklearn.cluster import SpectralBiclustering
import itertools

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from matplotlib import colors

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import blockworld_helpers as utils
import drawing_utils as drawing
import importlib
import scoring
import rda

In [2]:
## directory & file hierarchy
proj_dir = os.path.abspath('../..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(analysis_dir,'results')
stim_dir = os.path.join(proj_dir,'stimuli')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [3]:
iteration_name = 'Exp2Pilot3'
num_trials = 24 #for sanity checks

In [4]:
# Data already compiled into dataframes in CogSci 2020 Dataframe Generator

# trial_end data
trial_path = os.path.join(csv_dir,'block_silhouette_{}_good.csv'.format(iteration_name))
df = pd.read_csv(trial_path)

# # initial_block data
initial_path = os.path.join(csv_dir,'block_silhouette_initial_{}_good.csv'.format(iteration_name))
dfi = pd.read_csv(initial_path)

# # settled_block data
settled_path = os.path.join(csv_dir,'block_silhouette_settled_{}_good.csv'.format(iteration_name))
dfs = pd.read_csv(settled_path)

# # Sanity Check- same participants in each dataset.
df_participants = df.gameID.unique()
dfs_participants = dfs.gameID.unique()
assert Counter(df_participants) == Counter(dfs_participants)

n_before_outliers = len(df_participants)
print(str(n_before_outliers) + ' participants total')

49 participants total


In [6]:
def convert_to_str(flat_world):
    s = [str(i) for i in list(flat_world)] 
    res = "".join(s)
    return res

In [7]:
targets = np.sort(df['targetName'].unique())
ppts = np.sort(df['gameID'].unique())
reps = np.sort(df['repetition'].unique())

dfi['usableDiscreteWorld'] = dfi['discreteWorld'].apply(lambda a: 1+(-1)*np.array(ast.literal_eval(a)))
dfi['flatDiscreteWorld'] = dfi['discreteWorld'].apply(lambda a: (1+(-1)*np.array(ast.literal_eval(a))).flatten())

dfic = dfi[dfi.condition=='repeated']
dfic = dfic[['targetName','gameID','blockNum','repetition','flatDiscreteWorld']]
dfic['flatDiscreteWorldStr'] = dfic['flatDiscreteWorld'].apply(convert_to_str)

max_actions = dfic['blockNum'].max()

In [ ]:
# need an easy way of representing thickness of lines between two nodes

In [8]:
class Node:
    '''https://mail.google.com/mail/u/1/#inbox
    Represents a world state (as defined by discrete bitmap representation of blocks present, 
        as opposed to individual block dimensions and locations).
    Each Node can have multiple children.
    '''
    
    def __init__(self, flat_world):
        self.out_edges = [] #list of edges (child, ppt, rep)
        self.flat_world = flat_world 
        #self.flat_world_str = convert_to_str(flat_world)
        self.world_size = np.sum(flat_world)
        
    def get_ppts(self):
        self.out_edges.m
        
    def add_child(self):
        self.out_edges.m

In [9]:
class Edge:
    '''
    Edges contain child nodes, as well as the (ppt, repetition) pair that 
        uniquely defines a building path in the experiment (for one structure)
    '''
    def __init__(self, child, ppt, rep):
        self.child = child
        self.ppt = ppt
        self.rep = rep


In [10]:
class WorldLayer:
    '''
    Height in a tree is set by the sum of 'on' squares in the bitmap representation of the world.
    WorldLayer contains a dictionary that maps a flatdiscreteworld to a Node object
    '''
    def __init__(self, num_squares):
        self.num_squares = num_squares
        self.nodes = {}
        
    def get_world_node(self, flat_world, ppt, rep):
        '''
        add world state into layer
        if already exists, then returns the existing node.
        otherwise, creates new node and returns it
        
        '''
        flat_world_str = convert_to_str(flat_world)
        if flat_world_str in self.nodes:
            existing_node = self.nodes[flat_world_str]
            return existing_node
        else:
            new_node = Node(flat_world)
            nodes[flat_world_str] = new_node
            return new_node

In [16]:
dfic[(dfic.targetName==targets[0]) & (dfic.gameID==ppts[1]) & (dfic.repetition==0)]

,targetName,gameID,blockNum,repetition,flatDiscreteWorld,flatDiscreteWorldStr
171,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
172,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,2,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
173,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,3,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
174,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,4,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
175,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,5,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
176,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,6,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
177,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,7,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
178,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,8,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
179,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,9,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
180,hand_selected_004,0443-fd078367-c19d-495d-84d5-3519a281435d,10,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0000000000000000000000000000000000000000000000...
